In [1]:
import os
import pyspark
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, ArrayType
from pyspark.sql.functions import udf, regexp_replace
from pyspark.sql import types as T

In [2]:
try:
    sc.stop()
except:
    print("No SC connection")
try:
    spark.stop()
except:
    print("No spark connection")
try:
    spark.sparkContext.stop()
except:
    print("No spark.sparkContext connection")

In [125]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '12g'), 
                                   ('spark.executor.instances', '12'), 
                                   ('spark.executor.cores', '4'), 
                                   ('spark.yarn.executor.memoryOverhead', '2g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ('spark.driver.memory','10g'),
                                   ('spark.app.name','PS')])
sessn = SparkSession.builder.appName("PS").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

In [126]:
# Configuration
raw_table          = "pi_usrs.ps_837p_historical"
## Output table
preprocessed_table = "pi_usrs.js_ps_preprocessed_data"
model_data_table   = "pi_usrs.ps_history"

In [127]:
@udf("int")
def npi_validator(npi):
    """
    Valid NPI Checker, Rule for valid NPI: length of npi == 10, and starting digit should be 1 or 2.
    
    Parameter:
        npi (str).
        
    Returns:
        1 if the NPI is valid else False 0.
    """
    try:
        if (len(npi) == 10) & ((npi[0] == "2") | (npi[0] == "1")):
            return 1
        else:
            return 0
    except:
        return 0
    
@udf("int")
def place_of_srcv_validator(pls):
    """
    Check for valid place of service (00 to 99)
    
    Parameters:
        pls (string)
    
    Returns:
        1 if the place of service is valid else 0.
    """
    pls_lookup_list = ["0" + str(i) for i in range(1, 10)] + [str(i) for i in range(1,100)]
    
    try:
        if (pls in pls_lookup_list):
            return 1
        else:
            return 0
    except:
        return 0

In [128]:
def data_wrangling(data, col_to_trim):
    """
    This function includes, 
        ? Trimming of all the columns present in the data
        ? Casting of cl_line_item_chrg_amt & cl_unit_billed from StringType to DoubleType (Float).
        ? Creation of servicing_npi, rule -> When cl_rend_prov_npi is null or empty populate clm_bill_prov_npi otherwise populate cl_rend_prov_npi.
        ? Creation of cl_emergency_ind, rule -> When cl_emergency_ind =  "Y" populate 1 otherwise 0.
    Parameters:
        data (Pyspark Dataframe).
        col_to_trim (list):  List of columns      
    Returns:
        DataFrame (pySpark)
    """
    try:
        for col_ in col_to_trim:
            data = data.withColumn(col_, F.trim(data[col_]))
        data = data.withColumn("cl_line_item_chrg_amt", F.col("cl_line_item_chrg_amt").cast(DoubleType()))
        data = data.withColumn("cl_units_billed", F.col("cl_units_billed").cast(DoubleType()))
        data = data.withColumn("servicing_npi", 
                                       F.when(((F.col("cl_rend_prov_npi")=="") | F.col("cl_rend_prov_npi").isNull()),
                                              F.col("clm_bill_prov_npi")).otherwise(F.col("cl_rend_prov_npi")))
        data = data.withColumn("cl_emergency_ind", F.when(F.col("cl_emergency_ind") == "Y", 
                                                                  F.lit(1)).otherwise(F.lit(0)))
        return data
    except Exception as e:
        print(e)


In [129]:

def data_cleaning(data):
    """
    Data Cleaning is to identify and remove errors & duplicate data, in order to create a reliable dataset. This improves the quality of the data for analytics. Process includes,
    ? Dropping Claims having "NaN" and negative entry for 
        - cl_line_item_chrg_amt 
        - cl_unit_billed
    ? Dropping Claims having "Null" or "Blank" entries for 
        - servicing_npi
        - cl_proc_cd
        - Place of service
    ? Remove claims containing Place of service other than 01 to 99. 
    ? Remove claims containing NPI where the length is not equal to 10 digit or starting digit is not 1 nor 2.
    ? Remove claims containing modifiers which don’t have length  equal to 2.
    
    Parameters:
        data (Pyspark Dataframe) : Input Pyspark DataFrame
    """
    #Dropping claims having nan,  null or negative entry for cl_line_item_chrg_amt
    c = "cl_line_item_chrg_amt"
    print("Dropping claims having nan, null or negative entry for {}".format(c))
    # Filtering out garbage claims
    dump_data = data.filter((F.isnan(c) | F.col(c).isNull() | (F.col(c) < 0)))
    garbage_clms = dump_data.select("claim_number")
    # Adding flag for identification
    garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
    
    
    # Dropping claims having nan,  null or negative entry for cl_unit_billed
    c = "cl_units_billed"
    print("Dropping claims having nan, null or negative entry for {}".format(c))
    # Filtering out garbage claims
    dump_data = data.filter((F.isnan(c) | F.col(c).isNull() | (F.col(c) < 0)))
    garbage_clms1 = dump_data.select("claim_number")
    # Adding flag for identification
    garbage_clms1 = garbage_clms1.withColumn("Flag", F.lit(c))
    garbage_clms = garbage_clms.union(garbage_clms1)
    
    
    #Dropping claims having null or blank or where the length is not equal to 10 or starting digit is not 1 nor 2 for servicing_npi
    c = "servicing_npi"
    print("Dropping claims having null, blank or where length is not equal to 10 or starting digit is not 1 nor 2 for {}".format(c))
    # Running validation check for NPI
    data = data.withColumn("valid_npi", npi_validator(c)) 
    # Filtering out garbage claims
    dump_data = data.filter(((data[c] == "") | (data[c].isNull()) | (data["valid_npi"] == 0)))
    garbage_clms1 = dump_data.select("claim_number")
    # Adding flag for identification
    garbage_clms1 = garbage_clms1.withColumn("Flag", F.lit(c))
    garbage_clms = garbage_clms.union(garbage_clms1)
    
    
    # Dropping claims having "null" or "blank" & value other than 01 to 99 for cl_place_of_srvc
    c = "cl_place_of_srcv"
    # Creating the place of service lookup list having elements ranging from 01 to 99.
    global pls_lookup_list
    pls_lookup_list = ["0" + str(i) for i in range(1, 10)] + [str(i) for i in range(1,100)]
    print("Dropping claims having null or blank or value other than 01 to 99 for {}".format(c))
    # Running validation check for place of service
    data = data.withColumn("valid_pls", place_of_srcv_validator(c)) 
    # Filtering out garbage claims
    dump_data = data.filter(((data[c] == "") | (data[c].isNull()) | (data["valid_pls"] == 0)))
    garbage_clms1 = dump_data.select("claim_number")
    # Adding flag for identification
    garbage_clms1 = garbage_clms1.withColumn("Flag", F.lit(c))
    garbage_clms = garbage_clms.union(garbage_clms1)
    
    
    # Dropping claims where proc_mod_cd length is not equal to 2 
    # Generating the list of modifier columns
    mod_col_list = [col_ for col_ in data.columns if "proc_mod" in col_]
    for c in mod_col_list:
        print("Dropping claims where length of {} is not equal to 2". format(c))
        dump_data = data.filter((data[c] != "") & (~data[c].isNull()) & (F.length(F.col(c)) !=2))
        garbage_clms1 = dump_data.select("claim_number")
        # Adding flag for identification
        garbage_clms1 = garbage_clms1.withColumn("Flag", F.lit(c))
        garbage_clms = garbage_clms.union(garbage_clms1)
    
    
    # Dropping claims having null or blank entry for proc_cd
    c = "cl_proc_cd"
    print("Dropping claims having Null, blank entry for {}".format(c))
    # Filtering out garbage claims
    dump_data = data.filter(((data[c] == "")|(data[c].isNull())))
    garbage_clms1 = dump_data.select("claim_number")
    # Adding flag for identification
    garbage_clms1 = garbage_clms1.withColumn("Flag", F.lit(c))
    garbage_clms = garbage_clms.union(garbage_clms1)  
    
    ## Filtering out good claims
    garbage_claims = garbage_clms.groupBy("claim_number").agg(F.collect_set("Flag").alias("ps_discrepancy_flag"))
    
    clean_data = data.join(garbage_claims, on="claim_number", how ="left")
    clean_data = clean_data.filter(F.col("ps_discrepancy_flag").isNull())
    clean_data = clean_data.withColumn("cl_place_of_srcv", F.lpad(clean_data["cl_place_of_srcv"], 2, '0'))
    
    return clean_data

In [130]:
def get_mod_data(data, level, mod_col):
    """
    Aggregate the modifier data at claim line level; covert the column oriented modifier data to row oriented
    
    Parameters:
        data (Pysessn DataFrame)
        level (list): aggregation level
        mod_col (list): List of modifier columns
    
    Returns:
        data_mod (Pysessn DataFrame)
    """
    try:
        data_mod = data.select(level + mod_col)
        data_mod = data_mod.withColumn("cl_proc_mod_list", F.array(mod_col)).select(level + ["cl_proc_mod_list"])
        data_mod =  data_mod.withColumn("cl_proc_mod", F.explode("cl_proc_mod_list")).select(level + ["cl_proc_mod"])
        return data_mod
    except Exception as e:
        print(e)

def get_lookbck_data(modelling_data, base_data, lb_dur, entity, dt_col, grouping_entity):
    """
    
    Calculate the occurance probability of the entity of the provided lookback duration.

    Parameters:
        modelling_data: DataFrame (pySpark)
        base_data: DataFrame (pySpark)
        lb_dur: int
        entity: string
        dt_col: date object
        grouping_entity: string

    Returns:
        DataFrame (pySpark)
        
    """
    lst_dt_lb = dt_col + "_lst_dt_lb"
    lst_dt_m = dt_col + "_lst_dt_m"
    lb_col_list = [grouping_entity, entity]
    lb_col_list.append(dt_col)
    m_dt_data = modelling_data.select(dt_col).withColumn(lst_dt_m, F.last_day(F.from_unixtime(F.unix_timestamp(dt_col ,'yyyyMMdd'), 'yyyy-MM-dd')))
    m_dt_data = m_dt_data.select(lst_dt_m).distinct()
    df_mnth_list = []
    for m in range(lb_dur):
        m2 = -1*(m+1)
        lb_m = "mnth_diff_"+str(m+1)
        df_mnth_list.append(lb_m)
        m_dt_data = m_dt_data.withColumn(lb_m, F.add_months(F.col(lst_dt_m), m2))
    m_dt_data = m_dt_data.withColumn("temp", F.array(df_mnth_list))
    m_dt_data = m_dt_data.withColumn(lst_dt_lb, F.explode("temp")).select(lst_dt_m, lst_dt_lb)
    lb_dt_data = base_data.select(lb_col_list)
    lb_dt_data = lb_dt_data.withColumn(lst_dt_lb, F.last_day(F.from_unixtime(F.unix_timestamp(dt_col ,'yyyyMMdd'), 'yyyy-MM-dd')))
    lb_dt_data = lb_dt_data.drop(dt_col)
    lb_dt_data_freq = lb_dt_data.groupby(lb_dt_data.columns).count()
    md_data_freq = lb_dt_data_freq.join(m_dt_data, on = lst_dt_lb, how ="inner")
    md_data_freq = md_data_freq.groupby([grouping_entity, entity, lst_dt_m]).agg(F.sum("count").alias("freq"))
    md_data_freq_sum = md_data_freq.groupby([grouping_entity, lst_dt_m]).agg(F.sum("freq").alias("freq_sum"))
    md_data_freq_dist = md_data_freq.join(md_data_freq_sum, on = [grouping_entity, lst_dt_m], how = "inner")
    md_data_freq_dist = md_data_freq_dist.withColumn("freq_dist", F.round(F.col("freq")/F.col("freq_sum"), 3))
    lb_data = md_data_freq_dist.select(grouping_entity, lst_dt_m, entity, "freq_dist")
    lb_data = lb_data.withColumnRenamed(lst_dt_m , dt_col)
    return lb_data

In [ ]:
@udf(T.MapType(T.StringType(), T.FloatType()))
def create_struct( ):
    a = { }
    return a

In [8]:
data = sessn.sql("""select substring(trim(srk_clm_id),0,26) as claim_number,* from {raw}""".format(raw = raw_table))

In [9]:
data.agg(F.min("batch_run_dt").alias("Min Date"), F.max("batch_run_dt").alias("Max Date")).show()

+--------+--------+
|Min Date|Max Date|
+--------+--------+
|20190701|20200930|
+--------+--------+



In [10]:
data = data_wrangling(data, data.columns)

**Data Cleaning**

In [11]:
data = data_cleaning(data)

Dropping claims having nan, null or negative entry for cl_line_item_chrg_amt
Dropping claims having nan, null or negative entry for cl_units_billed
Dropping claims having null, blank or where length is not equal to 10 or starting digit is not 1 nor 2 for servicing_npi
Dropping claims having null or blank or value other than 01 to 99 cl_place_of_srcv
Dropping claims where length of cl_proc_mod1 is not equal to 2
Dropping claims where length of cl_proc_mod2 is not equal to 2
Dropping claims where length of cl_proc_mod3 is not equal to 2
Dropping claims where length of cl_proc_mod4 is not equal to 2
Dropping claims having Null, blank entry for cl_proc_cd


In [12]:
data.write.mode("overwrite").format("parquet").saveAsTable(preprocessed_table)

** Feature Engineering **

In [131]:
base_data = sessn.table(preprocessed_table)

In [132]:
base_data.select(F.min("batch_run_dt"), F.max("batch_run_dt")).show()

+-----------------+-----------------+
|min(batch_run_dt)|max(batch_run_dt)|
+-----------------+-----------------+
|         20190701|         20200930|
+-----------------+-----------------+



In [133]:
# Filtering out the lookback Table
modelling_data = base_data.filter(F.col("batch_run_dt")>="2019-12-01")

In [134]:
em_proc_code_list = ['99204', '99205', '99214', '99215', '99222', '99223', '99232', '99233']
high_cost_jcode_list = ['J9355', 'J1459', 'J0885', 'J0135', 'J0570', 'J0584', 'J1602', 'J9999']
group_level = ['servicing_npi', 'claim_number', 'fln_nbr']

In [108]:
# 1. Billed Amount
c = "billed_amt_sum"
print("Generating {}".format(c))
output_df = modelling_data.groupby(group_level).agg(F.sum("cl_line_item_chrg_amt").alias(c))

Generating billed_amt_sum


In [109]:
# 2. Number of claim lines
c = "clm_ln_cnt"
print("Generating {}".format(c))
clm_ln_cnt_data = modelling_data.groupby(group_level).agg(F.count("*").alias(c))
output_df = output_df.join(clm_ln_cnt_data, on = group_level, how ="left")

Generating clm_ln_cnt


In [110]:
# 3. Sum of Unit Billed
c = "clm_unit_billed"
print("Generating {}".format(c))
clm_unit_billed_data = modelling_data.groupby(group_level).agg(F.sum("cl_units_billed").alias(c))
output_df = output_df.join(clm_unit_billed_data, on = group_level, how ="left")

Generating clm_unit_billed


In [111]:
# 4. EM proc indicator 
c = "em_proc_ind"
print("Generating {}".format(c))
em_proc_code_list = [i.strip() for i in em_proc_code_list]
em_proc_code = sessn.createDataFrame(em_proc_code_list, StringType()).withColumnRenamed("value", "cl_proc_cd")
em_proc_code = em_proc_code.withColumn(c, F.lit(1))
em_proc_ind_data = modelling_data.join(em_proc_code, on ="cl_proc_cd", how="left")
em_proc_ind_data = em_proc_ind_data.fillna(0, subset = [c])
em_proc_ind_clm_data = em_proc_ind_data.groupby(group_level).agg(F.max(c).alias(c))
output_df = output_df.join(em_proc_ind_clm_data, on = group_level, how ="left")

Generating em_proc_ind


In [112]:
# 5.  high cost proc indicator 
c = "high_cst_proc_ind"
print("Generating {}".format(c))
high_cost_jcode_list = [i.strip() for i in high_cost_jcode_list]
high_cost_jcode = sessn.createDataFrame(high_cost_jcode_list, StringType()).withColumnRenamed("value", "cl_proc_cd")
high_cost_jcode = high_cost_jcode.withColumn(c, F.lit(1))
high_cst_proc_ind_data = modelling_data.join(high_cost_jcode, on ="cl_proc_cd", how="left")
high_cst_proc_ind_data = high_cst_proc_ind_data.fillna(0, subset=[c])
high_cost_proc_ind_clm_data = high_cst_proc_ind_data.groupby(group_level).agg(F.max(c).alias(c))
output_df = output_df.join(high_cost_proc_ind_clm_data, on = group_level, how ="left")

Generating high_cst_proc_ind


In [113]:
# 6. Emergency indicator 
c = "emgncy_ind"
print("Generating {}".format(c))
emrgncy_ind_data = modelling_data.withColumn(c, F.when(((F.col("cl_place_of_srcv")=="23")&(F.col("cl_emergency_ind")==0)),
                                                         F.lit(1)).otherwise(F.lit(0)))
emrgncy_ind_clm_data = emrgncy_ind_data.groupby(group_level).agg(F.max(c).alias(c))
output_df = output_df.join(emrgncy_ind_clm_data, on = group_level, how ="left")

Generating emgncy_ind


In [114]:
# Preparing data for Modifier signal
mod_col = [c[0] for c in modelling_data.dtypes if "mod" in c[0]]
data_mod = get_mod_data(modelling_data, group_level, mod_col)

In [115]:
# 7. Modifier 59 indicator 
c = "mod59_ind"
print("Generating {}".format(c))
data_mod_dummy = data_mod.select(group_level + ["cl_proc_mod"])
data_mod_dummy = data_mod_dummy.withColumn(c, F.when(F.col("cl_proc_mod")=="59", F.lit(1)).otherwise(F.lit(0)))
mod59_ind_clm_data = data_mod_dummy.groupby(group_level).agg(F.max(c).alias(c))
output_df = output_df.join(mod59_ind_clm_data, on = group_level, how ="left")

Generating mod59_ind


In [116]:
# 8. Modifier 91 indicator
c = "mod91_ind"
print("Generating {}".format(c))
data_mod_dummy = data_mod.select(group_level + ["cl_proc_mod"])
data_mod_dummy = data_mod_dummy.withColumn(c, F.when(F.col("cl_proc_mod")=="91", F.lit(1)).otherwise(F.lit(0)))
mod91_ind_clm_data = data_mod_dummy.groupby(group_level).agg(F.max(c).alias(c))
output_df = output_df.join(mod91_ind_clm_data, on = group_level, how ="left")

Generating mod91_ind


In [117]:
# 9. Modifier TC indicator - TW
c = "modTC_ind"
print("Generating {}".format(c))
data_mod_dummy = data_mod.select(group_level + ["cl_proc_mod"])
data_mod_dummy = data_mod_dummy.withColumn(c, F.when(F.col("cl_proc_mod")=="TC", F.lit(1)).otherwise(F.lit(0)))
modTC_ind_clm_data = data_mod_dummy.groupby(group_level).agg(F.max(c).alias(c))
output_df = output_df.join(modTC_ind_clm_data, on = group_level, how ="left")

Generating modTC_ind


In [118]:
# 10. Unusal procedure code
c = "proc_code_unusual"
lb_dur = 3
dt_col = "batch_run_dt"
entity = "cl_proc_cd"
grouping_entity = "servicing_npi"
lb_group_level = [grouping_entity, entity, dt_col]
# Generating Lookback lookup table
proc_code_lb_data = get_lookbck_data(modelling_data, base_data, lb_dur, entity, dt_col, grouping_entity)
# Merging the data 
modelling_data_lb = modelling_data.select(lb_group_level + ['claim_number', 'fln_nbr'])
modelling_data_lb = modelling_data_lb.withColumn("batch_run_dt", F.last_day(F.from_unixtime(F.unix_timestamp(F.col("batch_run_dt") ,'yyyyMMdd'), 'yyyy-MM-dd')))
proc_code_unusual_data = modelling_data_lb.join(proc_code_lb_data, on = lb_group_level, how = "left")
# Aggregating the result at claim level
proc_code_unusual_data = proc_code_unusual_data.fillna({"freq_dist":0})
proc_code_unusual_data = proc_code_unusual_data.groupby(['servicing_npi', 'claim_number', 'fln_nbr']).agg(F.min("freq_dist").alias(c))
output_df = output_df.join(proc_code_unusual_data, on = group_level, how ="left")

In [119]:
# 11. Unusal place of service
c = "pos_unsual"
lb_dur = 3
dt_col = "batch_run_dt"
entity = "cl_place_of_srcv"
grouping_entity = "servicing_npi"
lb_group_level = [grouping_entity, entity, dt_col]
# Generating Lookback lookup table
pos_lb_data = get_lookbck_data(modelling_data, base_data, lb_dur, entity, dt_col, grouping_entity)
# Merging the data 
modelling_data_lb = modelling_data.select(lb_group_level + ['claim_number', 'fln_nbr'])
modelling_data_lb = modelling_data_lb.withColumn("batch_run_dt", F.last_day(F.from_unixtime(F.unix_timestamp(F.col("batch_run_dt") ,'yyyyMMdd'), 'yyyy-MM-dd')))
pos_unusual_data = modelling_data_lb.join(pos_lb_data, on = lb_group_level, how = "left")
# Aggregating the result at claim level
pos_unusual_data = pos_unusual_data.fillna({"freq_dist":0})
pos_unusual_data = pos_unusual_data.groupby(['servicing_npi', 'claim_number', 'fln_nbr']).agg(F.min("freq_dist").alias(c))
output_df = output_df.join(pos_unusual_data, on = group_level, how ="left")

In [120]:
output_data = output_df.withColumn("batch_run_dt",F.substring(F.col("claim_number"),0,8))
output_data = output_data.withColumn("md" , lit(-999999.00) )
output_data = output_data.withColumn("dict" , create_struct( ) )
output_data = output_data.select('servicing_npi', 'claim_number', 'fln_nbr' , 'batch_run_dt', 'md' , 'dict', 'billed_amt_sum', 'clm_ln_cnt', 'clm_unit_billed', 'em_proc_ind', 'high_cst_proc_ind', 'emgncy_ind', 'mod59_ind', 'mod91_ind', 'modTC_ind', 'proc_code_unusual', 'pos_unsual')

In [121]:
', '.join(output_data.columns)

'servicing_npi, claim_number, billed_amt_sum, clm_ln_cnt, clm_unit_billed, em_proc_ind, high_cst_proc_ind, emgncy_ind, mod59_ind, mod91_ind, modTC_ind, proc_code_unusual, pos_unsual, batch_run_dt'

In [122]:
output_data.write.partitionBy("batch_run_dt").mode("overwrite").format("parquet").saveAsTable(model_data_table)

In [123]:
final_data = sessn.sql("select * from "+model_data_table)
print("Total Count : ",final_data.count())
final_data.head(n=10)

Total Count :  565969130


[Row(servicing_npi='1003002890', claim_number='202008261000000200000MIBST', billed_amt_sum=504.5, clm_ln_cnt=5, clm_unit_billed=8.0, em_proc_ind=0, high_cst_proc_ind=0, emgncy_ind=0, mod59_ind=0, mod91_ind=0, modTC_ind=0, proc_code_unusual=0.036, pos_unsual=0.886, batch_run_dt='20200826'),
 Row(servicing_npi='1003003070', claim_number='202008260030000600000MJOAD', billed_amt_sum=411.0, clm_ln_cnt=1, clm_unit_billed=1.0, em_proc_ind=1, high_cst_proc_ind=0, emgncy_ind=0, mod59_ind=0, mod91_ind=0, modTC_ind=0, proc_code_unusual=0.395, pos_unsual=0.772, batch_run_dt='20200826'),
 Row(servicing_npi='1003006214', claim_number='202008260350000700000MJHSC', billed_amt_sum=1883.0, clm_ln_cnt=3, clm_unit_billed=3.0, em_proc_ind=0, high_cst_proc_ind=0, emgncy_ind=0, mod59_ind=0, mod91_ind=0, modTC_ind=0, proc_code_unusual=0.111, pos_unsual=0.216, batch_run_dt='20200826'),
 Row(servicing_npi='1003009853', claim_number='202008260620000600000MIFW4', billed_amt_sum=159.0, clm_ln_cnt=3, clm_unit_bille

In [153]:
sessn.stop()